In [25]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error

In [26]:
test = pd.read_csv("test.csv")

In [27]:
train = pd.read_csv("train.csv")

In [28]:
stores = pd.read_csv("stores.csv")

In [29]:
transactions = pd.read_csv("transactions.csv")

In [30]:
holidays = pd.read_csv("holidays_events.csv")

In [31]:
oil = pd.read_csv("oil.csv")

In [32]:
oil.rename(columns={'dcoilwtico': 'oil_price'}, inplace=True)
train = pd.merge(train, oil, on='date', how='left')
test = pd.merge(test, oil, on='date', how='left')

train = pd.merge(train, stores, on='store_nbr', how='left')
test = pd.merge(test, stores, on='store_nbr', how='left')

condition = (holidays.transferred == True) | (holidays.type == 'Work Day')
special_days = holidays.drop(holidays[condition].index)
special_days = special_days.drop(['type', 'description', 'transferred'], axis=1)
special_days['special_days'] = 1
special_days = special_days.drop(special_days[special_days.date.duplicated()].index)

national = special_days[special_days.locale == 'National']
local = special_days[special_days.locale == 'Local']
regional = special_days[special_days.locale == 'Regional']

national = national.drop(['locale', 'locale_name'], axis=1)
train = pd.merge(train, national, on='date', how='left')
test = pd.merge(test, national, on='date', how='left')

for i, j in zip(regional.date, regional.locale_name):
    train['special_days'][(train.date == i) & (train.state == j)] = 1
for i, j in zip(regional.date, regional.locale_name):
    test['special_days'][(test.date == i) & (test.state == j)] = 1

for i, j in zip(local.date, local.locale_name):
    train['special_days'][(train.date == i) & (train.city == j)] = 1
for i, j in zip(local.date, local.locale_name):
    test['special_days'][(test.date == i) & (test.city == j)] = 1

/tmp/ipykernel_42696/690501391.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['special_days'][(train.date == i) & (train.state == j)] = 1
/tmp/ipykernel_42696/690501391.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['special_days'][(train.date == i) & (train.state == j)] = 1
/tmp/ipykernel_42696/690501391.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['special_days'][(train.date == i) & (train.state 

In [33]:
train.isnull().sum()

id                    0
date                  0
store_nbr             0
family                0
sales                 0
onpromotion           0
oil_price        928422
city                  0
state                 0
type                  0
cluster               0
special_days    2766390
dtype: int64

In [39]:
train['oil_price'].fillna(method='bfill', inplace=True)
test['oil_price'].fillna(method='bfill', inplace=True)

In [41]:
train['special_days'].fillna(0, inplace=True)
test['special_days'].fillna(0, inplace=True)

In [42]:
train.isnull().sum()

id              0
date            0
store_nbr       0
family          0
sales           0
onpromotion     0
oil_price       0
city            0
state           0
type            0
cluster         0
special_days    0
dtype: int64

In [43]:
train.set_index('id', inplace=True)
test.set_index('id', inplace=True)

In [45]:
train.drop(columns=['city', 'state', 'cluster', 'store_nbr'], axis=1, inplace=True)

KeyError: "['city', 'state', 'cluster', 'store_nbr'] not found in axis"

In [46]:
test.drop(columns=['city', 'state', 'cluster', 'store_nbr'], axis=1, inplace=True)